## Colab才需要用

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 35.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=2da7c08cf1e7b1a5e562fc9d21474be2933a95ca59fc2dd842d4521c1e4f84f4
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

# 建立資料

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

## 建立起始標籤表
- 客智科定期提供業務員標籤資料，寫入user區提供工程組使用
- 分別建立1/1、1/7的表

In [4]:
TAG_ID = ['100','200','100','200','300']
SAS_ID = ['A111','A111','B222','C333','D444']
NUM = ['1','3','3','2','1']
NUM_SCORE = []
VALID_FROM_DTTM = []

AG_TAG = {
    "TAG_ID": TAG_ID,
    "SAS_ID": SAS_ID,
    "NUM":NUM,
    "NUM_SCORE":np.nan,
    "VALID_FROM_DTTM":datetime(2021, 1, 1)
}

ag_tag_20210101 = pd.DataFrame(AG_TAG)

In [5]:
TAG_ID = ['100','100','200','200','100','300']
SAS_ID = ['A111','B222','B222','C333','D444','D444']
NUM = ['1','3','3','2','1','3']
NUM_SCORE = []
VALID_FROM_DTTM = []

AG_TAG = {
    "TAG_ID": TAG_ID,
    "SAS_ID": SAS_ID,
    "NUM":NUM,
    "NUM_SCORE":np.nan,
    "VALID_FROM_DTTM":datetime(2021, 1, 7)
}

ag_tag_20210107 = pd.DataFrame(AG_TAG)

In [6]:
print(ag_tag_20210101)
print(ag_tag_20210107)

  TAG_ID SAS_ID NUM  NUM_SCORE VALID_FROM_DTTM
0    100   A111   1        NaN      2021-01-01
1    200   A111   3        NaN      2021-01-01
2    100   B222   3        NaN      2021-01-01
3    200   C333   2        NaN      2021-01-01
4    300   D444   1        NaN      2021-01-01
  TAG_ID SAS_ID NUM  NUM_SCORE VALID_FROM_DTTM
0    100   A111   1        NaN      2021-01-07
1    100   B222   3        NaN      2021-01-07
2    200   B222   3        NaN      2021-01-07
3    200   C333   2        NaN      2021-01-07
4    100   D444   1        NaN      2021-01-07
5    300   D444   3        NaN      2021-01-07


In [7]:
ag_tag_20210101 = spark.createDataFrame(ag_tag_20210101)
ag_tag_20210107 = spark.createDataFrame(ag_tag_20210107)

## 比較兩張表
- 刪除A111的200標籤
- 新增B222的200標籤
- 修改D444的300標籤
- 新增D444的100標籤

In [8]:
ag_tag_20210101.show()
ag_tag_20210107.show()

+------+------+---+---------+-------------------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|
+------+------+---+---------+-------------------+
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|
+------+------+---+---------+-------------------+

+------+------+---+---------+-------------------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|
+------+------+---+---------+-------------------+
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|
|   300|  D444|  3|      NaN|2021-01-07 00:00:00|
+------+------+---+---------+-------------------+



## 寫入Hive的user區

In [9]:
# ag_tag_20210101.write.mode('overwrite').saveAsTable("cxl_user_9h000.ag_tag_20210101")
# ag_tag_20210107.write.mode('overwrite').saveAsTable("cxl_user_9h000.ag_tag_20210107")

# Initial Load 第一次資料歷程製作

In [10]:
from pyspark.sql.types import StringType, TimestampType, DataType
from pyspark.sql.functions import current_date, current_timestamp, to_timestamp, expr, to_date, col, lit

## 建立迄日
- 所有資料壓上迄日9999-12-31
- 排序資料檢查有無重複

In [11]:
ag_tag = ag_tag_20210101.withColumn("VALID_END_DTTM",lit("9999-12-31").cast(TimestampType())).sort(col("SAS_ID"),col("TAG_ID"),col("VALID_FROM_DTTM"))
ag_tag.show()

+------+------+---+---------+-------------------+-------------------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|
+------+------+---+---------+-------------------+-------------------+
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
+------+------+---+---------+-------------------+-------------------+



In [12]:
print("資料筆數：",ag_tag.count())
print("去除KEY重複筆數：",ag_tag.drop_duplicates(subset=['TAG_ID','SAS_ID','VALID_FROM_DTTM']).count())

資料筆數： 5
去除KEY重複筆數： 5


## 寫入Hive資料庫

In [13]:
# ag_tag.write.mode('overwrite').saveAsTable("cxl_user_9h000.ag_tag")

# Incremental load 資料更新歷程製作
- 讀取新資料表
- 抓取更新資料的最新日期，後面會用到

In [14]:
# ag_tag_n = spark.sql('''select * from cxl_user_9h000.ag_tag_20210107''')
ag_tag_n = ag_tag_20210107
etl_date = ag_tag_n.selectExpr("max(VALID_FROM_DTTM)").collect()[0][0]
print("資料更新時間:",etl_date)

資料更新時間: 2021-01-07 00:00:00


## 合併表個排序
- 讀取原資料表
- 新資料表迄日壓9999-12-31
- 合併兩張表並排序

In [15]:
# ag_tag = spark.sql('''select * from cxl_user_9h000.ag_tag''')
ag_tag.show()
ag_tag_n.show()

+------+------+---+---------+-------------------+-------------------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|
+------+------+---+---------+-------------------+-------------------+
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|
+------+------+---+---------+-------------------+-------------------+

+------+------+---+---------+-------------------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|
+------+------+---+---------+-------------------+
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|
|   100|  D444|  1|

In [16]:
from pyspark.sql.functions import desc, asc, lag, when, lead

- 新建DIFF_FLG作為邏輯判斷欄位
- 新表全壓I代表新增insert
- 舊表全壓UD代表可能會被刪除update2delete

In [17]:
ag_tag_n = ag_tag_n.withColumn("VALID_END_DTTM",lit("9999-12-31").cast(TimestampType())).withColumn("DIFF_FLG",lit("I"))
ag_tag_tmp = ag_tag.withColumn("DIFF_FLG",lit("UD")).union(ag_tag_n).sort(asc("SAS_ID"),asc("TAG_ID"),asc("VALID_FROM_DTTM"))

In [18]:
ag_tag_tmp.show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|999

## 建立資料更新標示
- 建立ID的partition並根據標籤及起日排序
- 每筆資料與後一筆相比，若標籤一樣、內容一樣，FLG標示0，代表不變保留
- 每筆資料與後一筆相比，若標籤一樣、內容不一樣，FLG標示U，代表更新update

In [21]:
from pyspark.sql.window import Window
w = Window.partitionBy("SAS_ID").orderBy(["TAG_ID","VALID_FROM_DTTM"])

In [22]:
ag_tag_tmp = ag_tag_tmp.withColumn("DIFF_FLG",when(
    (col("TAG_ID")==lead("TAG_ID",1).over(w)) & (col("NUM")==lead("NUM",1).over(w)) & (col("NUM_SCORE")==lead("NUM_SCORE",1).over(w)),"0"
    ).when(
    (col("TAG_ID")==lead("TAG_ID",1).over(w)) & ((col("NUM")!=lead("NUM",1).over(w)) | (col("NUM_SCORE")!=lead("NUM_SCORE",1).over(w))),"U"
    ).otherwise(col("DIFF_FLG")))
ag_tag_tmp.show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|999

- B222有兩個100標籤，起日不一樣，要刪除重複的新資料
- 每筆資料與前一筆相比，前筆FLG標示為0，則該筆資料為舊資料，重新標示D，代表delete

In [23]:
ag_tag_tmp = ag_tag_tmp.withColumn("DIFF_FLG",when(lag("DIFF_FLG",1).over(w)=="0","D").otherwise(col("DIFF_FLG")))
ag_tag_tmp.show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|999

## 壓資料更新迄日
- 更新的資料(FLG=U)與後一筆資料相比，壓後一筆資料起日的前一天
- 刪除的資料(FLG=UD)壓今天的更新日期

In [24]:
ag_tag_tmp = ag_tag_tmp.withColumn("VALID_END_DTTM",when(col("DIFF_FLG")=="U",(lead("VALID_FROM_DTTM",1).over(w))-expr("INTERVAL 1 seconds")).otherwise(col("VALID_END_DTTM")))
ag_tag_tmp.show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|      UD|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|202

In [25]:
ag_tag_tmp = ag_tag_tmp.withColumn("VALID_END_DTTM",when(col("DIFF_FLG")=="UD",etl_date-expr("INTERVAL 1 seconds")).otherwise(col("VALID_END_DTTM")))
ag_tag_tmp.show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  C333|  2|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  A111|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       D|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|2021-01-06 23:59:59|      UD|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|202

## 產出結果並寫入Hive

In [26]:
ag_tag_tmp.filter(ag_tag_tmp.DIFF_FLG != "D").sort(["SAS_ID","TAG_ID","VALID_FROM_DTTM"]).show()

+------+------+---+---------+-------------------+-------------------+--------+
|TAG_ID|SAS_ID|NUM|NUM_SCORE|    VALID_FROM_DTTM|     VALID_END_DTTM|DIFF_FLG|
+------+------+---+---------+-------------------+-------------------+--------+
|   100|  A111|  1|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  A111|  3|      NaN|2021-01-01 00:00:00|2021-01-06 23:59:59|      UD|
|   100|  B222|  3|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   200|  B222|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   200|  C333|  2|      NaN|2021-01-01 00:00:00|9999-12-31 00:00:00|       0|
|   100|  D444|  1|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
|   300|  D444|  1|      NaN|2021-01-01 00:00:00|2021-01-06 23:59:59|       U|
|   300|  D444|  3|      NaN|2021-01-07 00:00:00|9999-12-31 00:00:00|       I|
+------+------+---+---------+-------------------+-------------------+--------+



In [ ]:
# ag_tag_tmp.write.mode('overwrite').saveAsTable("cxl_user_9h000.ag_tag_n")